In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from scipy.signal import butter, lfilter
import os

In [80]:
def load_data_from_directory(directory):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path) 
            
            if 'velocity(m/s)' in df.columns:
                data = df['velocity(m/s)'].values 
            else:
                print(f"'velocity(m/s)' column not found in {filename}. Skipping this file.")
                continue
            
            data = (data - np.mean(data)) / np.std(data)
            
            data = np.expand_dims(data, axis=-1)
            
            all_data.append(data)
    
    all_data = np.concatenate(all_data, axis=0)
    return all_data

In [81]:
directory = "D:\\space_apps_2024_seismic_detection\\space_apps_2024_seismic_detection\\data\\lunar\\test\\data\\S12_GradeB"
all_data = load_data_from_directory(directory=directory)

df = pd.DataFrame(all_data, columns=['velocity(m/s)'])
df['velocity(m/s)'] = np.abs(df['velocity(m/s)'])
df

,velocity(m/s)
0,0.002413
1,0.002388
2,0.002363
3,0.002338
4,0.002312
...,...
36071392,0.002324
36071393,0.002323
36071394,0.002322
36071395,0.002322


In [82]:
print(df['velocity(m/s)'].isna().sum())
print(df.dtypes)

0
velocity(m/s)    float64
dtype: object


In [83]:
time_step = 0.15
sampling_rate = 1/time_step
sampling_rate

6.666666666666667

In [84]:
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist 
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

df['velocity(m/s)'] = butter_bandpass_filter(df, 0.5, 3, sampling_rate, order=7)
df


,velocity(m/s)
0,0.001076
1,0.001065
2,0.001053
3,0.001042
4,0.001031
...,...
36071392,0.001036
36071393,0.001036
36071394,0.001035
36071395,0.001035


In [85]:
velocity_fft = np.fft.fft(df['velocity(m/s)'])
frequencies = np.fft.fftfreq(len(df['velocity(m/s)']), time_step)

In [87]:
filtered_fft = df.copy()

In [88]:
filtered_fft['power'] = (velocity_fft ** 2) + (filtered_fft)

,velocity(m/s),velocity_fft,frequencies,power
0,0.001076,9.376268e+06+2.129354e- 10j,0.000000e+00,8.791440e+13
1,0.001065,1.761235e+05-4.339108e+ 05j,1.848186e-07,2.192981e+11
2,0.001053,-2.960789e+05+1.787155e+ 05j,3.696373e-07,1.196020e+11
3,0.001042,1.395049e+05-2.074472e+ 05j,5.544559e-07,6.249594e+10
4,0.001031,-1.454074e+05+1.040015e+ 05j,7.392746e-07,3.195962e+10
...,...,...,...,...
36071392,0.001036,-1.078817e+05-4.367575e+ 04j,-9.240932e-07,1.354603e+10
36071393,0.001036,-1.454074e+05-1.040015e+ 05j,-7.392746e-07,3.195962e+10
36071394,0.001035,1.395049e+05+2.074472e+ 05j,-5.544559e-07,6.249594e+10
36071395,0.001035,-2.960789e+05-1.787155e+ 05j,-3.696373e-07,1.196020e+11


In [90]:
normalized_values = normalize(filtered_fft.values, 'l2', axis=0)
norm_df = pd.DataFrame(normalized_values, columns=filtered_fft.columns)
norm_df.head()

,velocity(m/s),frequencies,power
0,4.017241e-07,0.000000e+00,0.999790
1,3.975492e-07,1.598992e-11,0.002494
2,3.933742e-07,3.197983e-11,0.001360
3,3.891993e-07,4.796975e-11,0.000711
4,3.850243e-07,6.395966e-11,0.000363


In [91]:
sta_window_s = 30
lta_window_s = 900

sta_window = int(sta_window_s * sampling_rate)
lta_window = int(lta_window_s * sampling_rate)

sta_kernel = np.ones(sta_window) / sta_window
lta_kernel = np.ones(lta_window) / lta_window

sta = np.convolve(norm_df['velocity(m/s)'], sta_kernel, mode='same')
lta = np.convolve(norm_df['velocity(m/s)'], lta_kernel, mode='same')

sta_lta_ratio = sta/lta
sta_lta_ratio[lta == 0] = 0

norm_df['STA/LTA'] = sta_lta_ratio 
norm_df

,velocity(m/s),frequencies,power,STA/LTA
0,4.017241e-07,0.000000e+00,0.999790,0.383871
1,3.975492e-07,1.598992e-11,0.002494,0.387658
2,3.933742e-07,3.197983e-11,0.001360,0.391466
3,3.891993e-07,4.796975e-11,0.000711,0.395279
4,3.850243e-07,6.395966e-11,0.000363,0.399085
...,...,...,...,...
36071392,3.869624e-07,-7.994958e-11,0.000154,0.823130
36071393,3.867192e-07,-6.395966e-11,0.000363,0.815736
36071394,3.866264e-07,-4.796975e-11,0.000711,0.808388
36071395,3.865692e-07,-3.197983e-11,0.001360,0.800951


In [92]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, LSTM, RepeatVector, TimeDistributed

In [93]:
norm_df.shape

(36071397, 4)

In [94]:
class CNN_LSTM_Autoencoder:
    def __init__(self, time_steps, n_filters, kernel_size, pool_size, latent_dim, epochs, batch_size):
        self.time_steps = time_steps
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        self.latent_dim = latent_dim
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
    
    def build_model(self):
        # Input layer
        input_layer = Input(shape=(self.time_steps, 4))

        # Encoder (down-sampling)
        x = Conv1D(filters=self.n_filters[0], kernel_size=self.kernel_size, activation='relu', padding='same')(input_layer)
        x = MaxPooling1D(pool_size=self.pool_size, padding='same')(x)
        
        x = Conv1D(filters=self.n_filters[1], kernel_size=self.kernel_size, activation='relu', padding='same')(x)
        x = MaxPooling1D(pool_size=self.pool_size, padding='same')(x)

        x = LSTM(self.lstm_units, activation='relu', return_sequences=True)(encoded)

        encoded = RepeatVector(self.time_steps)(x)

        x = LSTM(self.lstm_units, activation='relu', return_sequences=True)(encoded)

        x = Conv1D(filters=self.n_filters[1], kernel_size=self.kernel_size, activation='relu', padding='same')(x)
        x = UpSampling1D(size=self.pool_size)(x)

        x = Conv1D(filters=self.n_filters[0], kernel_size=self.kernel_size, activation='relu', padding='same')(x)
        x = UpSampling1D(size=self.pool_size)(x)

        # Reconstruction layer
        decoded = Conv1D(filters=4, kernel_size=self.kernel_size, activation='sigmoid', padding='same')(x)

        self.autoencoder = Model(inputs=input_layer, outputs=decoded)
        self.autoencoder.compile(optimizer='adam', loss='mse')
    
    def train(self, data):
        # Train the autoencoder
        history = self.autoencoder.fit(data, data, epochs=self.epochs, batch_size=self.batch_size)
        return history

    def detect_anomalies(self, data, threshold_percentile=95):
        # Get the reconstruction error
        reconstructed_data = self.autoencoder.predict(data)
        reconstruction_loss = np.mean(np.abs(data - reconstructed_data), axis=1)

        # Determine the anomaly threshold
        threshold = np.percentile(reconstruction_loss, threshold_percentile)
        anomalies = reconstruction_loss > threshold

        return anomalies, threshold

In [95]:
time_steps = int(5*60 / time_step)
time_steps

2000

In [96]:
norm_data = norm_df.values 

num_features = norm_df.shape[1]

num_sequences = norm_data.shape[0] // time_steps

reshaped_data = norm_data[:num_sequences * time_steps].reshape((num_sequences, time_steps, num_features))

In [98]:
cnn_lstm_autoencoder = CNN_LSTM_Autoencoder(
    time_steps=2000,  
    n_filters=[32, 16],      
    kernel_size=3,           
    pool_size=2,            
    latent_dim=8,           
    lstm_units=64,          
    epochs=50,               
    batch_size=64            
)

cnn_lstm_autoencoder.build_model()
history = cnn_lstm_autoencoder.train(reshaped_data)
history = CNN_LSTM_Autoencoder.train(reshaped_data)
print(history.history)
CNN_LSTM_Autoencoder.autoencoder.save('CNN_LSTM_Autoencoder.keras')


Epoch 1/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.1164
Epoch 2/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0761
Epoch 3/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0717
Epoch 4/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0743
Epoch 5/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0752
Epoch 6/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0714
Epoch 7/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0751
Epoch 8/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0715
Epoch 9/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0739
Epoch 10/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0740
Epoch 11/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0732
Epoch 12/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0763
Epoch 13/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0739
Epoch 14/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0716
Epoch 15/50
564/564 ━━━━━━━━━━━━━━━━━━━━ 8